In [1]:
import pandas as pd
import numpy as np
#from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, BaggingClassifier, StackingClassifier
#from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score, make_scorer
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold


#from sklearn.model_selection import StratifiedKFold
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import category_encoders as ce

from imblearn.combine import SMOTETomek

In [2]:
train_values = pd.read_csv('train_values.csv')
train_labels = pd.read_csv('train_labels.csv')

train_values = train_values.applymap(lambda col: np.nan if col == 995 else col)

drop_vars = ['building_id',
             'has_superstructure_bamboo',                
             'has_superstructure_mud_mortar_brick',       
             'has_superstructure_adobe_mud',              
             'legal_ownership_status',                    
             'has_secondary_use',                         
             'has_superstructure_stone_flag',             
             'has_superstructure_rc_non_engineered',      
             'has_superstructure_rc_engineered',          
             'has_secondary_use_agriculture',             
             'has_superstructure_cement_mortar_stone',    
             'has_secondary_use_hotel',                   
             'has_superstructure_other',                
             'has_secondary_use_rental',                  
             'has_secondary_use_other',                   
             'has_secondary_use_industry',               
             'has_secondary_use_institution',             
             'has_secondary_use_school',                  
             'has_secondary_use_health_post',            
             'has_secondary_use_gov_office',          
             'has_secondary_use_use_police' ]


train_values = train_values.drop(drop_vars, axis = 1)

targ_enc = ce.TargetEncoder()
targ_enc.fit(train_values, train_labels['damage_grade'])
train_values = targ_enc.transform(train_values)

imp = IterativeImputer(max_iter=20, min_value = 0)
imp.fit(train_values)

train_values = pd.DataFrame(imp.transform(train_values), columns = train_values.columns)

/home/matt/anaconda3/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [9]:
dtrain_full = xgb.DMatrix(train_values, train_labels['damage_grade'].astype(int))

In [10]:
skf = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 23456)

In [11]:
xgb_params = {'objective': 'multi:softmax',
 'gamma': 0.001369048068388758,
 'eta': 0.11,
 'max_depth': 15,
 'min_child_weight': 9,
 'num_class' : 4,
 'subsample' : .85,
 'colsample_bytree' : .9
 }

In [ ]:
def f1_eval(y_pred, dtrain):
    y_true = dtrain.get_label()
    print(np.argmax(np.round(y_pred)))
    err = 1-f1_score(y_true, np.round(y_pred), average = 'micro')
    return 'f1_err', err

In [ ]:
cv_results = xgb.cv(xgb_params,
                    dtrain_full,
                    num_boost_round = 999,
                    seed = 234,
                    folds = skf,
                    early_stopping_rounds = 10,
                    feval = f1_eval
                    )
print(cv_results)
print(1 - cv_results['test-merror-mean'].min())

In [26]:
f1_scores = []
boost_rounds = []
for train_idx, test_idx in skf.split(train_values, train_labels.damage_grade):
    dtrain = dtrain_full.slice(train_idx)
    dtest  = dtrain_full.slice(test_idx)
    
    model = xgb.train(xgb_params,
                      dtrain,
                      num_boost_round = 999,
                      evals = [(dtest, 'test_set')],
                      early_stopping_rounds = 10,
                      verbose_eval = False
                  )
    boost_rounds.append(model.best_iteration + 1)
    print("Boost Rounds: {}".format(model.best_iteration + 1))
    
    preds = model.predict(dtest)
    
    f1_scores.append(f1_score(dtest.get_label(), preds, average = 'micro'))

print(f1_scores)
print("Mean F1: {}".format(np.mean(f1_scores)))
print("Mean Boost Rounds: {}".format(np.mean(boost_rounds)))

94
133
99
118
124


In [29]:
print(f1_scores)
print(np.mean(f1_scores))
print(np.mean([94, 133, 99, 118, 124]))

[0.7444791926478771, 0.7457981580966999, 0.7474865694551035, 0.7425748273215657, 0.744071373752878]
0.7448820242548249
113.6


In [30]:
best_model = xgb.train(xgb_params, dtrain_full, num_boost_round = 114)

In [33]:
test = pd.read_csv('test_values.csv')
mtest = test.drop(drop_vars, axis = 1)
mtest = targ_enc.transform(mtest)
mtest = pd.DataFrame(imp.transform(mtest), columns = mtest.columns)
dtest = xgb.DMatrix(mtest)

test["damage_grade"] = best_model.predict(dtest)
test["damage_grade"] = test["damage_grade"].astype(int)
print(test.value_counts('damage_grade')) # check that preds look ok

damage_grade
2    56282
3    24302
1     6284
dtype: int64


In [34]:
test[['building_id', 'damage_grade']].to_csv("xgb_model.csv", index = False)